In [4]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install protobuf==3.20.0
!pip install konlpy
!pip install kobert_transformers
!pip install JPype1
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://github.com/SKTBrain/KoBERT.git to /private/var/folders/k0/mjg221ts4hlcztnwnqx_7nm40000gn/T/pip-install-n681xqnp/kobert-tokenizer_81e91bf422b545378090908f1e46404d
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /private/var/folders/k0/mjg221ts4hlcztnwnqx_7nm40000gn/T/pip-install-n681xqnp/kobert-tokenizer_81e91bf422b545378090908f1e46404d
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /private/var/folders/k0/mjg221ts4hlcztnwnqx_7nm40000gn/T/pip-req-build-48lniktq
  Running command git clone --filter=blob:none --quiet 'https://****@github.com/SKTBrain/KoBERT.git' /private/var/folders/k0/mjg221ts4hlcztnwnqx_7nm40000gn/T/pip-req-build-48lniktq
  Resolved https://****@github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing

In [95]:

# KoBERT 모델 로딩 및 설정
import torch
import numpy as np
import mxnet as mx
from transformers import AutoModel
from kobert_transformers import get_tokenizer
from kobert import get_mxnet_kobert_model

tokenizer = get_tokenizer()
model, vocab = get_mxnet_kobert_model(use_decoder=False, use_classifier=False)



def getKeywordFromSentence(inputText):
    
    # 예시 문장 생성
    sentence = inputText


    # 입력 데이터 전처리 및 추론
    tokens = tokenizer.tokenize(sentence)
    input_ids = torch.Tensor(tokenizer.convert_tokens_to_ids(tokens)).to(torch.int).unsqueeze(0)


    print(input_ids)
    
    token_type_ids = np.zeros(len(input_ids))
    input_ids_np = np.array(input_ids, dtype=np.int32)
    token_type_ids = np.array(token_type_ids, dtype=np.int32)

    # numpy array를 NDArray로 변환
    input_ids_mx = mx.nd.array(input_ids_np, dtype=np.int32)
    token_type_ids = mx.nd.array(token_type_ids, dtype=np.int32)

    encoder_layer, pooled_output = model(input_ids_mx, token_type_ids)
    last_hidden_states = encoder_layer[0]
    
    numpy_array = last_hidden_states.asnumpy()
    last_hidden_states = torch.tensor(numpy_array)
    
    print(last_hidden_states.shape)
    
    

    # 출력 확인

    # 각 토큰의 중요도를 계산합니다.
    hidden_transpose = torch.transpose(last_hidden_states, 0, 1)
    importance = torch.sum(hidden_transpose, dim=0)

    print(importance.shape)
    # 중요도가 높은 순서대로 정렬하여 상위 5개의 단어를 추출합니다.
    num_keywords = 5
    importance_with_index = [(x, i) for i, x in enumerate(importance)]
    importance_with_index.sort(reverse = False)
    top_indices = [input_ids[0][i] for x, i in importance_with_index]
    top_indices = torch.tensor(top_indices)
    print(top_indices)
    


    keywords = [tokenizer.convert_ids_to_tokens(torch.unsqueeze(top_indices[i],0))[0] for i in range(num_keywords)]

    print(keywords)

    return keywords

using cached model. /Users/gimjinnyeong/Documents/capstone/backend/nlp/.cache/mxnet_kobert_45b6957552.params
using cached model. /Users/gimjinnyeong/Documents/capstone/backend/nlp/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [96]:
result = getKeywordFromSentence ("""
[토스뱅크] 자동이체(송금) 약관 개정 안내
안녕하세요. 토스뱅크입니다.
항상 토스뱅크를 이용해 주시는 고객님께 진심으로 감사드립니다.

토스뱅크 『자동이체(송금) 약관』이 다음과 같이 개정될 예정이오니 이용에 참고하여 주시기 바랍니다.

1. 대상 약관 : 자동이체(송금) 약관

2. 시행일 : 2023년 4월 20일

3. 기존 고객 적용 여부 : 적용

4. 『자동이체(송금) 약관』 변경 주요 내용

""")

print(result)

tensor([[ 702, 7628, 6664, 6321, 7565,  363, 3892, 7096, 7436,   18, 6621, 5550,
           40, 3211, 5474,  851, 3136, 3135, 5724, 7814,   54, 4737, 6664, 6321,
         7565, 7139,   54, 4996, 4737, 6664, 6321, 7565, 6116, 3730, 4213, 6705,
         5760,  994, 5778, 5611, 4360, 6745, 7078,  793, 5925,   54, 4737, 6664,
         6321, 7565,  517, 5238, 7147, 5872, 7096, 7436,   18, 6621, 5550,   40,
         3211, 5474, 5239, 7096, 1577, 5468,  834,  851, 5902, 3413, 7096, 6964,
         5770, 3726, 6896, 4430, 7815, 4213, 6705, 5561, 2186, 6019, 5772,   54,
          529,   54, 1651, 3211, 5474,  629, 3892, 7096, 7436,   18, 6621, 5550,
           40, 3211, 5474,  553,   54, 3004, 7126,  629,  554,  135, 5712,  605,
         7028,  554, 7126,  589,   54, 1300,  994, 4004, 3310,  629, 4004,  605,
           54,  517, 5238, 7147, 5872, 7096, 7436,   18, 6621, 5550,   40, 3211,
         5474, 5239, 2340, 4233, 1449]], dtype=torch.int32)
torch.Size([137, 768])
torch.Size([137])
tensor([